In [1]:
from Bio import SeqIO

# Global variables for accessing datasets
directory = "/home/callum/rotation2/ops_aligment/ncbi_dataset/data"

# Note GCF is the refseq data, GCA in the genbank data
pst_directory = directory + "/GCF_000007805.1/"
psp_directory = directory + "/GCF_000012205.1/"
pss_directory = directory + "/GCF_000012245.1/"
pao1_directory = directory + "/GCF_000006765.1/"
pa14_directory = directory + "/GCF_000014625.1/"

def region_extract(organism, region_start, region_end, output_suffix):
    if organism == "pst":
        global pst_directory
        input_file = pst_directory
        # Target record_id for chromosome
        target_record_id = "NC_004578.1"
        plasmids = True
    elif organism == "pss":
        global pss_directory
        input_file = pss_directory
        # Target record_id for chromosome
        target_record_id = "NC_007005"
        plasmids = False
    elif organism == "psp":
        global psp_directory
        input_file = psp_directory
        # Target record_id for chromosome
        target_record_id = "NC_005773.3"
        plasmids = True
    elif organism == "pao1":
        global pao1_directory
        input_file = pao1_directory
        # Target record_id for chromosome
        target_record_id = "NC_002516"
        plasmids = False
    elif organism == "pa14":
        global pa14_directory
        input_file = pa14_directory
        # Target record_id for chromosome
        target_record_id = "CP000438"
        plasmids = False
    else:
        UserWarning("Organism is not supported")
    # Defining filepaths
    input_file = input_file + "genomic.gbff"
    output_file = organism + "_" + output_suffix
    if not plasmids:
        # Using SeqIO.read as only one locus in the sequence (no plasmids)
        record = SeqIO.read(input_file, "genbank")
        sub_record = record[region_start:region_end]
        SeqIO.write(sub_record, output_file, "genbank")
    else:
        with open(input_file) as handle:
            for record in SeqIO.parse(handle, "genbank"):
                #print(record.id)
                if record.id == target_record_id:
                    sub_record = record[region_start:region_end]
                    sub_record.id = f"{record.id}_{region_start}_{region_end}"
                    sub_record.description = f"{record.description} (region {region_start}-{region_end})"
                    sub_record.annotations = record.annotations
                    sub_record.features = [
                        f for f in record.features
                        if f.location.start < region_end and f.location.end > region_start
                    ]
                    SeqIO.write(sub_record, output_file, "genbank")
                    break

In [ ]:
# Extracting genes of interest for systematic proteome searches
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

# --- Settings ---
input_gbff = "your_input_file.gbff"   # replace with your actual .gbff file
output_fasta = "genes_by_locus_tag.fasta"
locus_tags_to_extract = {"PSPTO_0182", "LpxA", "LpxC"}  # <-- Set of locus tags you want

# --- Extraction ---
records_to_write = []

for record in SeqIO.parse(input_gbff, "genbank"):
    for feature in record.features:
        if feature.type == "CDS":
            locus_tag = feature.qualifiers.get("locus_tag", [None])[0]
            if locus_tag in locus_tags_to_extract:
                seq = feature.extract(record.seq)
                rec = SeqRecord(seq, id=locus_tag, description="")
                records_to_write.append(rec)

# --- Write to FASTA ---
SeqIO.write(records_to_write, output_fasta, "fasta")

print(f"Extracted {len(records_to_write)} gene(s) to {output_fasta}")



In [ ]:
# Extracting putative CPA cluster genes

# Region around main CPA locus (ABC transporters)
region_extract("psp",1110824, 1186815,"CPA01.gbk")

region_extract("pst",1145640,1226640,"CPA01.gbk")


region_extract("pss",1010000, 1080555,"CPA01.gbk")

region_extract("pao1",6141666-35000, 6141666+35000,"CPA01.gbk")
region_extract("pa14",6414586-35000, 6414586+35000,"CPA01.gbk")
# Region around gmd in DC3000

region_extract("pst",1098690-35000,1098690+35000,"CPA02.gbk")



In [3]:
# Extracting putative OSA cluster genes
# Using buffer region of 35000 bp from start of WbpL
buffer = 35000
# Region around main CPA locus (ABC transporters)
start_coord = 1926688
region_extract("pst",start_coord - buffer,start_coord + buffer,"OSA01.gbk")

start_coord = 4200515
region_extract("psp",start_coord - buffer,start_coord + buffer,"OSA01.gbk")

start_coord = 4319067 
region_extract("pss",start_coord - buffer,start_coord + buffer,"OSA01.gbk")

start_coord = 3530467 
region_extract("pao1",start_coord - buffer,start_coord + buffer,"OSA01.gbk")
start_coord = 2038087 
region_extract("pa14",start_coord - buffer,start_coord + buffer,"OSA01.gbk")


In [14]:
# Extracting putative Alginate cluster genes
# Using buffer region of 20000 bp from start of AlgA
buffer = 20000
# Region around main CPA locus (ABC transporters)
start_coord = 1351082
region_extract("pst",start_coord - buffer,start_coord + buffer,"Alg01.gbk")

start_coord = 1302146
region_extract("psp",start_coord - buffer,start_coord + buffer,"Alg01.gbk")

start_coord = 1189612 
region_extract("pss",start_coord - buffer,start_coord + buffer,"Alg01.gbk")

In [ ]:
# Extracting putative Psl cluster genes
# Using buffer region of 20000 bp from start of PslB
buffer = 20000
# Region around main CPA locus (ABC transporters)
start_coord = 3983868
region_extract("pst",start_coord - buffer,start_coord + buffer,"Psl01.gbk")

start_coord = 3734971
region_extract("psp",start_coord - buffer,start_coord + buffer,"Psl01.gbk")

start_coord = 3951223 
region_extract("pss",start_coord - buffer,start_coord + buffer,"Psl01.gbk")

# pst bit around 3447
3889623 

In [8]:
# Bash commands to run to generate the html
conda activate clinker
clinker *CPA*gbk -p CPA.html
explorer.exe CPA.html


SyntaxError: invalid syntax (2668342951.py, line 2)

In [ ]:
input_file = pst_directory+"genomic.gbff"
output_file = "pst_region2.gbk"

# Replace with your target contig ID (e.g., 'NC_002516.2' for P. aeruginosa chromosome)

#region_start, region_end = 1145640, 1186640
region_start, region_end = 1145640, 1226640



with open(input_file) as handle:
    for record in SeqIO.parse(handle, "genbank"):
        if record.id == target_record_id:
            sub_record = record[region_start:region_end]
            sub_record.id = f"{record.id}_{region_start}_{region_end}"
            sub_record.description = f"{record.description} (region {region_start}-{region_end})"
            sub_record.annotations = record.annotations
            sub_record.features = [
                f for f in record.features
                if f.location.start < region_end and f.location.end > region_start
            ]
            SeqIO.write(sub_record, output_file, "genbank")
            break

In [ ]:
directory = "/home/callum/rotation2/ops_aligment/ncbi_dataset/data"

input_file = psp_directory+"genomic.gbff"
output_file = "psp_region2.gbk"

# Replace with your target contig ID (e.g., 'NC_002516.2' for P. aeruginosa chromosome)

#region_start, region_end = 1120824, 1156815
region_start, region_end = 1110824, 1186815

with open(input_file) as handle:
    for record in SeqIO.parse(handle, "genbank"):
        if record.id == target_record_id:
            sub_record = record[region_start:region_end]
            sub_record.id = f"{record.id}_{region_start}_{region_end}"
            sub_record.description = f"{record.description} (region {region_start}-{region_end})"
            sub_record.annotations = record.annotations
            sub_record.features = [
                f for f in record.features
                if f.location.start < region_end and f.location.end > region_start
            ]
            SeqIO.write(sub_record, output_file, "genbank")
            break

In [ ]:
directory = "/home/callum/rotation2/ops_aligment/ncbi_dataset/data"
pss_directory = directory + "/GCA_000012245.1/"
input_file = pss_directory+"genomic.gbff"
output_file = "pss_region2.gbk"

# Replace with your target contig ID (e.g., 'NC_002516.2' for P. aeruginosa chromosome)
target_record_id = "NC_007005.1"
#region_start, region_end = 1010000, 1054555
region_start, region_end = 1010000, 1080555

# Using SeqIO.read as only one locus in the sequence (no plasmids)
record = SeqIO.read(input_file, "genbank")
sub_record = record[region_start:region_end]
SeqIO.write(sub_record, output_file, "genbank")

1